First, we need to read the data. I'll load the HEF and header data into pandas dataframes.

In [1]:
import os
import h5py
import numpy as np
import pandas as pd
from ntplib import system_to_ntp_time

pd.set_option('display.notebook_repr_html', True)

NTP_DIFF = system_to_ntp_time(0)

hef_file = os.path.join('data', 'horizontal_electric_field.nc')
header_file = os.path.join('data', 'hpies_data_header.nc')

def netcdf_to_pandas(path):
    h5file = h5py.File(path)
    times = h5file['time'].value
    times = times - NTP_DIFF
    times = pd.to_datetime(times, unit='s')
    data = {}
    for group in h5file:
        if group == 'time':
            continue
        for var in h5file[group]:
            if 'timestamp' in var:
                continue
            data[var] = h5file[group][var].value
    return pd.DataFrame(data, index=times)
    
hef_df = netcdf_to_pandas(hef_file)
header_df = netcdf_to_pandas(header_file)

Now, we can check the contents.

In [2]:
len(hef_df), len(header_df)

(12009, 252)

We have lots of HEF data and header data at around 1/50th the data rate. Let's re-index the header data to the same time series as the HEF data, forward filling the missing data points (using the last seen value).

In [3]:
header_df = header_df.reindex(index=hef_df.index, method='ffill')
len(header_df)

12009

Now the time series for both datasets match, let's join the two dataframes:

In [4]:
joined = hef_df.join(header_df, rsuffix='_header')
len(joined)

12009

Now joined contains the combination of the HEF data and header data. We now have a hpies_hcno value for all data points in the HEF data:

In [5]:
joined[['hpies_eindex', 'hpies_e1a', 'hpies_e1b', 'hpies_e1c', 'hpies_e2a', 'hpies_e2b', 'hpies_e2c', 'hpies_hcno']]

,hpies_eindex,hpies_e1a,hpies_e1b,hpies_e1c,hpies_e2a,hpies_e2b,hpies_e2c,hpies_hcno
2015-04-29 14:21:08.658715,9,396607,396698,396062,396132,397092,396776,0
2015-04-29 14:21:08.708307,19,397149,397297,396651,396784,397725,397392,0
2015-04-29 14:21:08.766100,29,397208,397333,396677,396802,397759,397383,0
2015-04-29 14:21:08.825862,39,397201,397312,396647,396779,397729,397419,0
2015-04-29 14:21:08.887132,49,397217,397346,396732,396766,397747,397432,0
2015-04-29 14:21:08.947684,59,397223,397290,396708,396768,397735,397399,0
2015-04-29 14:21:09.009053,69,397214,397339,396698,396777,397755,397416,0
2015-04-29 14:21:09.069960,79,397198,397299,396715,396769,397741,397386,0
2015-04-29 14:21:09.131373999,89,397185,397311,396714,396771,397748,397420,0
2015-04-29 14:21:09.191834,99,397210,397304,396700,396760,397751,397413,0


Let's compute the data rate for the HEF data:

In [6]:
data_rate = 1 / np.median(np.diff(joined.index.values).astype('f64') / 1e9)
'data rate: %.2f Hz' % data_rate

'data rate: 16.32 Hz'

Now, let's compute the storage requirements for the lifetime of the project if we are to add just one single 32-bit integer to each HEF particle:

In [7]:
one_day = data_rate * 60 * 60 * 24
one_year = one_day * 365
project_lifetime = one_year * 25
int_storage = project_lifetime * 4 / 1e9
'Expected storage cost for adding one int to each particle: %.2f GB' % int_storage 

'Expected storage cost for adding one int to each particle: 51.48 GB'

So, by storing the header data separately and putting the onus on the user to combine the datasets we save over 51 GB (just for the hcno data alone.)

Now, let's see if we can figure out what went wrong to produce the occasional jump in eindex.

In [8]:
import ntplib
import struct
import time
import os

message_types = ( 'Data From Instrument',
                  'Data From Driver',
                  'Port Agent Command',
                  'Port Agent Status',
                  'Port Agent Fault',
                  'Instrument Command',
                  'Heartbeat',
                  'Pickled Data From Instrument',
                  'Pickled Data From Driver' )

HEADER_FORMAT = '>3sBHHII'
SYNC_BYTES = '\xa3\x9d\x7a'
raw_data_fh = open(os.path.join('data', 'hpies-port_agent.20150429.data'))
raw_data = raw_data_fh.read()

In [9]:
def ntp_time_to_ascii(tstamp):
    return time.asctime(time.gmtime(ntplib.ntp_to_system_time(tstamp)))

def parse_header(data):
    sync, message_type, packet_size, checksum, time_upper, time_lower = struct.unpack_from(HEADER_FORMAT, data)
    timestamp = time_upper + float(time_lower) / 2**32
    return message_type, packet_size, checksum, timestamp

Data from the instrument is message type 1

In [10]:
HEADER_SIZE = struct.calcsize(HEADER_FORMAT)
packets = []
while raw_data:
    start = raw_data.find(SYNC_BYTES)
    if start == -1:
        break
    
    message_type, packet_size, checksum, timestamp = parse_header(raw_data[start:])
    packet = raw_data[start:start+packet_size]
    raw_data = raw_data[start+packet_size:]
    
    if message_type == 1:
        packets.append((timestamp, packet))

In [11]:
import sys
NEWLINE = '\r\n'
records = []
buf = ''

for index, packet in enumerate(packets):
    timestamp, data = packet
    if index == 0:
        tstamp = timestamp
        

    buf += data[HEADER_SIZE:]
    if NEWLINE in buf:
        n_index = buf.index(NEWLINE)
        records.append((tstamp, buf[:n_index]))
        buf = buf[n_index+len(NEWLINE):]
        tstamp = timestamp

In [12]:
HEF_records = [r for r in records if r[1].startswith('#3__DE')]
times = np.array([r[0] for r in HEF_records]) - NTP_DIFF
times = pd.to_datetime(times, unit='s')
raw_hef_df = pd.DataFrame([r[1] for r in HEF_records], index=times)
len(raw_hef_df) - len(joined)

24

There are 24 more records in the source data than the parsed data. This matches the number of jumps:

In [13]:
eindex = joined.hpies_eindex.values
jumps = np.diff(eindex) == 20
len(eindex[jumps])

24

so, what's up with these records?

Let's build a new jumps indexed to the raw data by inserting another item after each jump...

In [14]:
new_jumps = []
for jump in jumps:
    new_jumps.append(jump)
    if jump:
        new_jumps.append(True)
        
new_jumps.append(False)
        
pd.set_option('max_colwidth',100)
jump_df = raw_hef_df[new_jumps]
jump_df

,0
2015-04-29 15:33:09.045122,#3__DE 59 396663 397170 397282 397417 396834 397734*2a72
2015-04-29 15:33:09.106457,#3__DE 69 396672 397189 397303 397438 396820 397744\r*23f7
2015-04-29 15:51:09.297572,#3__DE 89 396670 397219 397320 397410 396820 397780*2f49
2015-04-29 15:51:09.363066,#3__DE 99 396608 397176 397380 397444 396801 397774\r*f8f6
2015-04-29 15:59:08.987504,#3__DE 39 396702 397227 397332 397406 396827 397772*0f21
2015-04-29 15:59:09.123707,#3__DE 49 396688 397234 397354 397451 396786 397788*8ebd
2015-04-29 16:05:09.367980,#3__DE 89 396630 397215 397357 397455 396827 397783*de97
2015-04-29 16:05:09.500727999,#3__DE 99 396668 397191 397333 397413 396832 397757*c653
2015-04-29 16:15:12.684828,#3__DE 629 396631 397200 397350 397442 396814 397777*f296
2015-04-29 16:15:12.870204,#3__DE 639 396700 397181 397355 397443 396812 397769*ba65


In [15]:
joined.hpies_eindex[jumps]

2015-04-29 15:33:09.045122        59
2015-04-29 15:51:09.302055        89
2015-04-29 15:59:08.991716        39
2015-04-29 16:05:09.367980        89
2015-04-29 16:15:12.688811       629
2015-04-29 16:15:13.975050       829
2015-04-29 16:19:10.132692       209
2015-04-29 16:19:12.551835       599
2015-04-29 16:29:14.309592       889
2015-04-29 16:37:13.759633       779
2015-04-29 16:45:10.409904       259
2015-04-29 16:53:09.252755999     49
2015-04-29 16:59:09.398863        89
2015-04-29 17:17:11.879393       489
2015-04-29 17:19:11.738710       459
2015-04-29 17:39:14.174500       849
2015-04-29 17:43:09.832230       139
2015-04-29 18:05:13.977414       799
2015-04-29 18:39:14.808507       929
2015-04-29 18:53:09.611510        99
2015-04-29 18:53:11.228877       349
2015-04-29 19:03:09.612759        99
2015-04-29 19:23:09.425629        59
2015-04-29 19:23:12.258533       529
Name: hpies_eindex, dtype: int64

These seem to line up as expected.

We've already identified this data as problematic for the parser:

`#3__DE 69 396672 397189 397303 397438 396820 397744\\r*23f7`

We'll need to check why this record is failing:

`#3__DE 49 396688 397234 397354 397451 396786 397788*8ebd`

In [16]:
def crc3kerm(buf):
    """
    Compute the Kermit checksum on @a buf
    """
    crcta = [0, 4225, 8450, 12675, 16900, 21125, 25350, 29575,
             33800, 38025, 42250, 46475, 50700, 54925, 59150, 63375]
    crctb = [0, 4489, 8978, 12955, 17956, 22445, 25910, 29887,
             35912, 40385, 44890, 48851, 51820, 56293, 59774, 63735]
    crc = 0
    for i in range(0, len(buf)):
        c = crc ^ ord(buf[i])
        hi4 = (c & 240) >> 4
        lo4 = c & 15
        crc = (crc >> 8) ^ (crcta[hi4] ^ crctb[lo4])
    return crc


def chksumnmea(s):
    xor = 0
    for i in range(0, len(s)):
        xor ^= ord(s[i])
    return xor

def good_checksum(data):
    response, crc_str = raw.split('*')
    crc = int(crc_str, 16)
    calc_crc = crc3kerm(response)
    return crc == calc_crc

raw = jump_df.iloc[5].values[0]
good_checksum(raw)

True

CRC is good... so why did this fail?

Do we have any checksum failures, maybe my indexing is wrong...

In [17]:
[each[0] for each in raw_hef_df.values if not good_checksum(each[0])]

[]

Nope, all checksums pass...